# ENCODER

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from transformers import AutoModel, AutoTokenizer



In [ ]:
tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')
model = AutoModel.from_pretrained('ai4bharat/indic-bert')

In [ ]:
import torch
import torch.nn.functional as F

#Mean Pooling - Take average of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
#Encode text
def encode(texts):
    # Tokenize sentences
    doc_stride = 128
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512, stride=doc_stride, return_overflowing_tokens = True)
    encoded_input.pop("overflow_to_sample_mapping")

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings.tolist()

# MYSQL

# get data

In [1]:
import pandas as pd


In [2]:
fp = r"..\dataset\created_ta.csv"
qa_dataset = pd.read_csv(fp)

In [17]:
qa_dataset.drop(["Unnamed: 0"], axis = 1, inplace=True)

In [20]:
qa_dataset.to_json("created_tqa.json", force_ascii=False)

In [42]:
qa_dataset

,context,question,answer_text,answer_start
0,கணினியின் தேவை தற்போதைய காலங்களில் மிகவும் அதி...,எதனுடைய தேவை தற்போதைய காலங்களில் மிகவும் அதிகர...,கணினியின் தேவை,0
1,கணினியின் தேவை தற்போதைய காலங்களில் மிகவும் அதி...,எது இல்லாத இடமே தற்போது இல்லை?,கணினி,0
2,கணினியின் தேவை தற்போதைய காலங்களில் மிகவும் அதி...,மனித சமூகம் எதை நம்பி தன் வாழ்க்கையை ஒப்படைத்த...,கணினியை நம்பி,168
3,கணினியின் தேவை தற்போதைய காலங்களில் மிகவும் அதி...,கணினி தனது சேவையை எதில் செய்கிறது?,"வேளாண்மை, தொழில்நுட்பவியல், பொறியியல், மருத்து...",258
4,மனித சமூகம் தகவல்களை சேமிக்க தொடங்கிய நாள் முத...,எத்தனை வருடங்களுக்கு முன்னதாக இருந்த கணினியை வ...,20,155
...,...,...,...,...
263,சுந்தரர் இயற்றியது திருத்தொண்டர்த் தொகை.நம்பிய...,சுந்தரர் இயற்றிய நூலின் பெயர் என்ன?,திருத்தொண்டர்த் தொகை,19
264,சுந்தரர் இயற்றியது திருத்தொண்டர்த் தொகை.நம்பிய...,திருத்தொண்டத் திருவந்தாதியின் ஆசிரியர் யார்?,நம்பியாண்டார் நம்பியால்,40
265,சுந்தரர் இயற்றியது திருத்தொண்டர்த் தொகை.நம்பிய...,பெரிய புராணத்தை இயற்றியவர் யார்?,சேக்கிழாரால்,148
266,சுந்தரர் இயற்றியது திருத்தொண்டர்த் தொகை.நம்பிய...,மீனாட்சி சுந்தரனார் பெரிய புராணத்தின் ஆசிரியரை...,"""பக்திச் சுவை நனி சொட்டச் சொட்டப் பாடிய கவி வலவ""",308


In [4]:
rows = []
for index, row in qa_dataset[['context', 'question', 'answer_text', 'answer_start']].iterrows():
    rows.append({
            'context': row['context'],
            'question': row['question'],
            'answer_text': row['answer_text'],
            'answer_start': row['answer_start'],
            })

In [43]:
rows

[{'context': 'கணினியின் தேவை தற்போதைய காலங்களில் மிகவும் அதிகரித்துள்ளது. அரசு அலுவலகம் தனியார் அலுவலகம் வங்கிகள் கல்லூரிகள் பள்ளிகள் என கணினி இல்லாத இடமே தற்போது இல்லை. மனித சமூகம் கணினியை நம்பி தன் வாழ்க்கையை ஒப்படைத்து பல வருடங்கள் ஆகிவிட்டன. மனிதனுக்கு உதவும் வகையில் வேளாண்மை, தொழில்நுட்பவியல், பொறியியல், மருத்துவம், ராணுவம், வானியல் ஆராய்ச்சி என அனைத்து இடங்களிலும் கணினி தனது சேவையை செய்கிறது.',
  'question': 'எதனுடைய தேவை தற்போதைய காலங்களில் மிகவும் அதிகரித்துள்ளது?',
  'answer_text': 'கணினியின் தேவை',
  'answer_start': 0},
 {'context': 'கணினியின் தேவை தற்போதைய காலங்களில் மிகவும் அதிகரித்துள்ளது. அரசு அலுவலகம் தனியார் அலுவலகம் வங்கிகள் கல்லூரிகள் பள்ளிகள் என கணினி இல்லாத இடமே தற்போது இல்லை. மனித சமூகம் கணினியை நம்பி தன் வாழ்க்கையை ஒப்படைத்து பல வருடங்கள் ஆகிவிட்டன. மனிதனுக்கு உதவும் வகையில் வேளாண்மை, தொழில்நுட்பவியல், பொறியியல், மருத்துவம், ராணுவம், வானியல் ஆராய்ச்சி என அனைத்து இடங்களிலும் கணினி தனது சேவையை செய்கிறது.',
  'question': 'எது இல்லாத இடமே தற்போது இல்லை?',
  'answer_te

In [44]:
import json

In [45]:
fp = "new.json"
file = open(fp,"w", encoding="utf8")

file.write(json.dumps(rows, ensure_ascii=False, indent=4))
file.close()

In [6]:
import pymysql
conn = pymysql.connect(host='localhost', user='aswin', port=3306, password='Mysql@123', database='ODQA',local_infile=True)
cursor = conn.cursor()
print("odqa")
# print("this is imp")
TABLE_NAME = 'QA_DATASET'

odqa


In [7]:
def create_context_table():
    #Deleting previouslny stored table for clean run
    drop_table = "DROP TABLE IF EXISTS " + TABLE_NAME + ";"
    cursor.execute(drop_table)
    try:
        # sql = "CREATE TABLE if not exists " + TABLE_NAME + " (id TEXT, context TEXT);"
        sql = f"""
                CREATE TABLE if not exists {TABLE_NAME} (
                    id int(10) NOT NULL AUTO_INCREMENT,
                    question TEXT COLLATE utf8_bin NOT NULL,
                    context MEDIUMTEXT COLLATE utf8_bin NOT NULL,
                    answer  TEXT COLLATE utf8_bin NOT NULL,
                    answer_start int(5) NOT NULL,
                    PRIMARY KEY (id)
                ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_bin
                AUTO_INCREMENT=1 ;"""
        cursor.execute(sql)
        print(f"{TABLE_NAME} table successfully!")
    except Exception as e:
        print("can't create a MySQL table: ", e)



In [8]:
create_context_table()

QA_DATASET table successfully!


In [50]:
def execute_query(query):
    try:
        cursor.execute(query)
        rows = cursor.fetchall()
        return rows
    except Exception as e:
        print("can't create a MySQL table: ", e)



In [10]:
id = 1
q = f"select context from QA_DATASET where id = {id}"
q = f"select * from context where id between 0 and 10 ;"
res = execute_query(q)

NameError: name 'execute_query' is not defined

In [13]:
def insert_data(dataset):
    """
    context should be array of contexts
    [con1, con2, ...]
    """
    # q = "select count(id) from context"
    # res = execute_query(q)
    # current_size = res[0][0]
    # next = current_size+1
    for data in dataset:
        sql = "INSERT INTO QA_DATASET (question, context, answer, answer_start) VALUES (%s, %s, %s, %s)"
        cursor.execute(sql, (data["question"], data["context"], data["answer_text"], data["answer_start"]))
        # next+=1 
    conn.commit()

def extract_context(id):
    q = f"select context from QA_DATASET where id = {id}"
    res = execute_query(q)
    return res[0]

In [11]:
rows[0]

{'context': 'கணினியின் தேவை தற்போதைய காலங்களில் மிகவும் அதிகரித்துள்ளது. அரசு அலுவலகம் தனியார் அலுவலகம் வங்கிகள் கல்லூரிகள் பள்ளிகள் என கணினி இல்லாத இடமே தற்போது இல்லை. மனித சமூகம் கணினியை நம்பி தன் வாழ்க்கையை ஒப்படைத்து பல வருடங்கள் ஆகிவிட்டன. மனிதனுக்கு உதவும் வகையில் வேளாண்மை, தொழில்நுட்பவியல், பொறியியல், மருத்துவம், ராணுவம், வானியல் ஆராய்ச்சி என அனைத்து இடங்களிலும் கணினி தனது சேவையை செய்கிறது.',
 'question': 'எதனுடைய தேவை தற்போதைய காலங்களில் மிகவும் அதிகரித்துள்ளது?',
 'answer_text': 'கணினியின் தேவை',
 'answer_start': 0}

In [14]:
insert_data(rows)


# MILVUS

In [53]:

import odqa_mysql as odqa_mysql
import odqa_encoder as odqa_encoder

mysql connection established
Retriever model loaded


In [46]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility
connections.connect()

In [47]:
from tqdm.autonotebook import tqdm

In [48]:
import json

TABLE_NAME = 'question_answering'
collection = None



In [49]:
#Deleting previouslny stored table for clean run
def create_mqa():
    if utility.has_collection(TABLE_NAME):
        collection = Collection(name=TABLE_NAME)
        collection.drop()

    field1 = FieldSchema(name="ind", dtype=DataType.INT64, descrition="int64", is_primary=True)
    field2 = FieldSchema(name="id", dtype=DataType.INT64, descrition="int64", is_primary=False)
    field3 = FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, descrition="float vector",dim=1024, is_primary=False)
    schema = CollectionSchema(fields=[field1, field2, field3], description="collection description")
    collection = Collection(name=TABLE_NAME, schema=schema)
    
    default_index = {"index_type": "IVF_FLAT", "metric_type": 'IP', "params": {"nlist": 200}}
    collection.create_index(field_name="embedding", index_params=default_index)
    database_handler = {"info": "index should start from 1", "sqlrows": 368, "milvus_rows": 0, "milvus_index": 1, "batch": 10}
    db_fp = r"database_handler.json"
    file = open(db_fp,"w")
    json.dump(database_handler, file)
    file.close()
    

if utility.has_collection(TABLE_NAME):
    collection = Collection(name=TABLE_NAME)

In [50]:
def push_context_to_milvus():
    print("\n\n")
    db_fp = r"database_handler.json"
    file = open(db_fp)
    database_handler = json.loads(file.read())
    file.close()

    start= database_handler['milvus_rows']
    end= start+database_handler["batch"]
    index = database_handler['milvus_index']
    print(index)
    
    query = f"select * from context where id between {start} and {end} ;"
    res = odqa_mysql.execute_query(query)

    for id, context in tqdm(res):
        emb = odqa_encoder.encode(context)
        indexs = []
        ids = []
        for i in range(len(emb)):
            indexs.append(index)
            index+=1  
            ids.append(id)
        # print(emb, indexs, ids)
        collection.insert([indexs, ids, emb])
           
    database_handler['milvus_rows'] = end
    database_handler['milvus_index'] = index-1

    file = open(db_fp,"w")
    json.dump(database_handler, file)
    file.close()
    
    mysql_size = odqa_mysql.execute_query("select count(*) from QA_DATASET")[0][0]
    return f"mysql : {mysql_size}\nmilvus : {collection.num_entities}"

In [51]:
create_mqa()

In [55]:
push_context_to_milvus()




11


100%|██████████| 11/11 [00:23<00:00,  2.16s/it]


'mysql : 268\nmilvus : 21'

In [ ]:
search_params = {"metric_type": "IP", "params": {"nprobe": 10}}

def find_similar(emb):
    collection.load()
    return collection.search(
	data=emb, 
	anns_field="embedding", 
	param=search_params, 
	limit=10, 
	output_fields = ['id', 'ind'], 
	expr=None,
	consistency_level="Strong"
)


In [ ]:
search_param = {
  "data": val,
  "anns_field": "embedding",
  "param": {"metric_type": "L2", "params": {"nprobe": 10}},
  "limit": 2,
  "expr": None,
  "output_fields" : ['id'] 
}
res = collection.search(**search_param)

In [ ]:
len(val[0])

1024

In [ ]:
res = find_similar(val)

In [ ]:
res

In [ ]:
res

In [ ]:
res[0]

In [ ]:
for re in res:
    print(re)

['(distance: 40.82012176513672, id: 0)', '(distance: 42.13190460205078, id: 8)']


In [58]:
def query():
    collection.load()
    return collection.query(
	anns_field="embedding", 
	param={"metric_type": "L2", "params": {"nprobe": 10}},
	output_fields = ["id"] ,
	limit=10, 
	expr="id < 20",
	consistency_level="Strong"
)

In [59]:
res = query()

In [60]:
res

[{'ind': 1, 'id': 1},
 {'ind': 2, 'id': 2},
 {'ind': 4, 'id': 4},
 {'ind': 10, 'id': 10},
 {'ind': 12, 'id': 11},
 {'ind': 15, 'id': 14},
 {'ind': 16, 'id': 15},
 {'ind': 18, 'id': 17},
 {'ind': 19, 'id': 18},
 {'ind': 11, 'id': 10},
 {'ind': 13, 'id': 12},
 {'ind': 14, 'id': 13},
 {'ind': 17, 'id': 16},
 {'ind': 20, 'id': 19},
 {'ind': 3, 'id': 3},
 {'ind': 5, 'id': 5},
 {'ind': 6, 'id': 6},
 {'ind': 7, 'id': 7},
 {'ind': 8, 'id': 8},
 {'ind': 9, 'id': 9}]

# EXTRACTOR

In [ ]:
from transformers import  pipeline

# model_name = "deepset/xlm-roberta-large-squad2"
model_name = "AswiN037/xlm-roberta-squad-tamil"

answer_extract = pipeline('question-answering', model=model_name, tokenizer=model_name)


# Evaluation

In [ ]:
from datasets import load_metric

In [ ]:
rouge = load_metric("rouge")

In [ ]:
# !pip install absl-py
# !pip install rouge_score


In [ ]:
res = [['206', ' 206'],
 ['காசுமீரில்', ' காசுமீரில்'],
 ['சர் அலெக்ஸாண்டர் ஃபிளெமிங்', ' அலெக் ஸாண்டர் ஃப்பௌமிங்.'],
 ['தாலாட்டு', '\n(ஆயர்பாடி…)'],
 ['சூரியனும்', ' சூரியன்'],
 ['IBM', ' IBM'],
 ['அலெக்ஸாண்டர் கிரகாம் பெல்', ' பெல்'],
 ['1914ம்', ' (1914'],
 ['அக்டோபர் 12, 1993', ' அக்டோபர் 12, 1993[1]'],
 ['27', ' 27'],
 ['உருகுவே', ' பிரேசில்'],
 ['பிரான்ஸ்', ' பிரான்ஸ்'],
 ['30,368,609', '\n(km²)'],
 ['185 மீட்டர்', ' 185 மீட்டர்'],
 ['கி.பி.1510', '\nகி.பி.1510-ம்'],
 ['கி.மு. ஐந்தாம் நூற்றாண்டில்', ' (கி.மு 470/469'],
 ['60 நொடிகள்', ' 60'],
 ['பசுபிக்', ' பசுபிக்'],
 ['லிஸ்பன்', ' லிஸ்பன்'],
 ['உடல் திசு ஆய்வு', ' (Men Get Breast Cancer'],
 ['சசி', ' சசி'],
 ['1,376  கிலோ மீட்டர்', ' 1370 கிமீ'],
 ['1000', ' 1025நீர்  1000ஈத்தைல்'],
 ['மூங்கில்', ' மூங்கிலால்'],
 ['புளோரிடாவில்', ' புளோரிடாவில்'],
 ['208', ' 206'],
 ['இந்தியத் திரைப்பட இசைப் பாடகர்', ' பின்னணிப்பாடகர்'],
 ['5488', ' 5488'],
 ['சனவரி 28, 1892', ' 1995'],
 ['அலெக்ஸாண்டர்-எட்மண்ட பெக்கெரெலின்', ' ஆர்க்கிமிடீஸ்'],
 ['அகுவாபா', ' அகுவாபா'],
 ['பிரிட்டனில்', ' ஸ்பெயினில்,'],
 ['புதன்', ' (Mercury)'],
 ['நெமடோடா', '\nஎக்டிசாசோவாக்கள்'],
 ['26 மே 2001', ' 26 மே 2001'],
 ['8 லட்சத்துக்கு', ' 8 லட்சத்துக்கு மேற்பட்டவை'],
 ['அ. இர. ரகுமான்', ' நியமிக்கப்பட்டார்.[5]'],
 ['தொற்று', '\nதொற்று'],
 ['தமிழ்', ' தமிழ்'],
 ['Hibiscus rosa-sinensis', ' செவ்வரத்தை'],
 ['கரிகாலன்', ' கரிகாலன்'],
 ['சுவீடனில்', ' சுவீடனில்'],
 ['மேரிகியூரி மற்றும் பியரிகியூரி தம்பதியரால்',
  ' மேரிகியூரி மற்றும் பியரிகியூரி'],
 ['10,911', ' 4,694 மீட்டர்.'],
 ['பசிபிக் பெருங்கடலாகும்', ' அத்திலாந்திக் பெருங்கடல்'],
 ['புதன்', ' வாகும்.'],
 ['செவுள்கள், நுரையீரலால்', ' (நுரையீரல்'],
 ['1,568.7 square kilometres', ' 1,568.7 square kilometres'],
 ['தோக்கியோ', ' தோக்கியோ'],
 ['1000', ' 1000'],
 ['விஷம்', ' விஷம் கொடுத்துக்'],
 ['மெசொப்பொத்தேமியர்கள்', ' எகிப்தியர்கள்'],
 ['பிடி', ' பிடி'],
 ['ஈராக்', ' Mesopotamia.'],
 ['சனவரி 26ஆம்', ' குடியரசு நாள் அன்று'],
 ['ஆறாம் முகம்மது', '\nமுதலாம் சுலைமானின்'],
 ['செவ்வாய்', ' செவ்வாயின்'],
 ['1901', ' 1901'],
 ['29', ' 14'],
 ['வெளிப்பரப்பு அடர்த்தியான சாம்பல் நிறத்திலும், உள்நிறை மஞ்சள், வெள்ளை',
  ' சாம்பல்'],
 ['1623', ' 27 அக்டோபர் 1605),[3][4]'],
 ['பைக்கால்', ' பைக்கால்'],
 ['அவுஸ்திரேலியா', ' ஆஸ்திரேலிய'],
 ['ஒழுங்கற்ற', ' இரண்டாவது சிறிய'],
 ['மாட்ரிட்', ' மாட்ரிட்.'],
 ['என்றிகோ பெர்மியின்', ' என்றிகோ பெர்மியின்'],
 ['70 ஆண்டுகள்', ' 70 ஆண்டுகள்).'],
 ['பெங்களூரு', ' பெங்களூரில்'],
 ['ஆறு', ' ஆறு'],
 ['ஐந்தில் ஒரு பங்கு', ' ஐந்தில் ஒரு பங்கு'],
 ['1982', ' 1982ம்'],
 ['ஆல்ஃபிரட் நோபல்', ' ஆல்ஃபிரட் நோபெல்'],
 ['தென் அமெரிக்க', ' தென் அமெரிக்க'],
 ['சாக்ரடீசு', ' பிளேட்டோவும்'],
 ['பழங்குடி மொழி', ' பழங்குடி மொழி'],
 ['சர்தார் வல்லப்பாய் படேல்', ' சர்தார் வல்லப்பாய் படேல்'],
 ['70', ' 70 ஆண்டுகள்).'],
 ['7870', ' 7870வெள்ளீயம்'],
 ['ஆர்கெண்ட்டம்', ' ஆர்கெண்ட்டம்'],
 ['டிசம்பர் 25, 1642', ' (டிசம்பர் 25, 1642'],
 ['கெய்ரோ', ' கெய்ரோ'],
 ['எட்டு கோள்களையும், ஐந்து குறுங்கோள்களையும்', ' எட்டு'],
 ['கொலம்பசு', ' கொலம்பஸ்'],
 ['2008', ' 1957'],
 ['350,000', ' (International Code of Nomenclature for Cultivated Plants)'],
 ['இரண்டு மில்லியன் சதுர கிலோமீட்டர்', '\n1,972,550 சதுர கிலோமீட்டர்'],
 ['1935', ' 1935'],
 ['3', ' 3'],
 ['ஜான் ஷெப்பர்ட் பேரோன்', ' ஜான் ஷெப்பர்ட் பேரோன்'],
 ['பிரெஞ்சு', ' பிரெஞ்சு'],
 ['24', ' 24']]

In [ ]:
# predictions = [res[0][0]]
# reference = [res[0][1]]
reference = []
predictions = []

for ref, pre in res:
    reference.append(ref)
    predictions.append(pre)


In [ ]:
len(predictions), len(reference)

(91, 91)

In [ ]:
rouge.compute(predictions=predictions, references=reference)

{'rouge1': AggregateScore(low=Score(precision=0.16112637362637366, recall=0.16483516483516483, fmeasure=0.1641025641025641), mid=Score(precision=0.24358974358974356, recall=0.25274725274725274, fmeasure=0.2468864468864469), high=Score(precision=0.3461996336996337, recall=0.3626373626373626, fmeasure=0.35239926739926736)),
 'rouge2': AggregateScore(low=Score(precision=0.005494505494505495, recall=0.01098901098901099, fmeasure=0.007326007326007326), mid=Score(precision=0.038461538461538464, recall=0.04395604395604396, fmeasure=0.04029304029304029), high=Score(precision=0.07706043956043944, recall=0.08791208791208792, fmeasure=0.08424908424908424)),
 'rougeL': AggregateScore(low=Score(precision=0.15929487179487184, recall=0.16483516483516483, fmeasure=0.16117216117216115), mid=Score(precision=0.24725274725274726, recall=0.25274725274725274, fmeasure=0.2490842490842491), high=Score(precision=0.32967032967032966, recall=0.34065934065934067, fmeasure=0.33342490842490835)),
 'rougeLsum': Aggr

# sample QA

In [23]:
import json

In [38]:
fp = r"..\dataset/created_tqa.json"
df = pd.read_json(fp)
from datasets import Dataset
df = Dataset.from_pandas(df)


In [41]:
df.to_json("sam.json", )

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]


TypeError: to_json() got an unexpected keyword argument 'ascii'

In [37]:
df[0]

{'context': 'கணினியின் தேவை தற்போதைய காலங்களில் மிகவும் அதிகரித்துள்ளது. அரசு அலுவலகம் தனியார் அலுவலகம் வங்கிகள் கல்லூரிகள் பள்ளிகள் என கணினி இல்லாத இடமே தற்போது இல்லை. மனித சமூகம் கணினியை நம்பி தன் வாழ்க்கையை ஒப்படைத்து பல வருடங்கள் ஆகிவிட்டன. மனிதனுக்கு உதவும் வகையில் வேளாண்மை, தொழில்நுட்பவியல், பொறியியல், மருத்துவம், ராணுவம், வானியல் ஆராய்ச்சி என அனைத்து இடங்களிலும் கணினி தனது சேவையை செய்கிறது.',
 'question': 'எதனுடைய தேவை தற்போதைய காலங்களில் மிகவும் அதிகரித்துள்ளது?',
 'answer_text': 'கணினியின் தேவை',
 'answer_start': 0,
 '__index_level_0__': 0}